In [1]:
%reload_ext autotime
import pandas as pd
import geopandas as gpd
from glob import glob
import os
from tqdm.auto import tqdm
import statsmodels.formula.api as smf
tqdm.pandas()

In [2]:
pd.set_option('display.max_columns', None)
df = pd.read_csv("results.csv")
LLM_results = pd.read_csv("LLM_results.csv")
new_cols = LLM_results.columns[~LLM_results.columns.isin(df.columns)].tolist()
df = df.merge(LLM_results[new_cols + ["panoid"]], on="panoid")
df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs="EPSG:4326").to_crs(2193)
df.to_csv("merged_results.csv", index=False)
df

,Index,pid,n,time,anxiousness,latitude,longitude,geometry,panoid,panolat,panolon,panodate,panothirdparty,source,uploader,green,environment,active_transport,quality,water,obscured,people,cars,bikes,dwellings,shops,offices
0,0,P20001,1,2023-04-25T02:51:42Z,0,-36.924795,174.738044,POINT (1754803.112 5912059.517),IvrcS0W1RlFAlnci-p39XA,-36.924665,174.737914,2012-04,False,launch,NaN,25,low density residential,False,60,0,0,0,2,0,7,2,0
1,1,P20001,2,2023-04-25T08:43:13Z,0,-36.924801,174.738076,POINT (1754806.011 5912058.736),IvrcS0W1RlFAlnci-p39XA,-36.924665,174.737914,2012-04,False,launch,NaN,25,low density residential,False,60,0,0,0,2,0,7,2,0
2,2,P20001,3,2023-04-26T02:22:26Z,0,-36.924783,174.738071,POINT (1754805.575 5912060.734),IvrcS0W1RlFAlnci-p39XA,-36.924665,174.737914,2012-04,False,launch,NaN,25,low density residential,False,60,0,0,0,2,0,7,2,0
3,3,P20001,4,2023-04-26T09:00:53Z,0,-36.924797,174.738123,POINT (1754810.215 5912059.092),IvrcS0W1RlFAlnci-p39XA,-36.924665,174.737914,2012-04,False,launch,NaN,25,low density residential,False,60,0,0,0,2,0,7,2,0
4,4,P20001,5,2023-04-26T20:36:19Z,3,-36.924771,174.738084,POINT (1754806.75 5912062.109),IvrcS0W1RlFAlnci-p39XA,-36.924665,174.737914,2012-04,False,launch,NaN,25,low density residential,False,60,0,0,0,2,0,7,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1318,1426,P20721,1,2024-05-05T02:00:52Z,1,-36.893455,174.728262,POINT (1753994.828 5915552.341),CfRtPfDMNhfXHTNvMwnYRw,-36.893394,174.728062,2024-06,False,launch,NaN,65,low density residential,False,60,200,20,1,2,0,3,1,0
1319,1427,P20721,2,2024-05-05T07:36:22Z,1,-36.893483,174.728253,POINT (1753993.98 5915549.306),CfRtPfDMNhfXHTNvMwnYRw,-36.893394,174.728062,2024-06,False,launch,NaN,65,low density residential,False,60,200,20,1,2,0,3,1,0
1320,1428,P20721,3,2024-05-05T23:06:27Z,2,-36.845252,174.759951,POINT (1756917.266 5920848.722),AF1QipN2FD2eYEmK8bRpEgoM7fFl5-nUstwWujnRj0gv,-36.845292,174.759939,2022-06-24,True,photos:street_view_publish_api,Mint Design,0,commercial,False,80,0,70,0,6,0,4,4,2
1321,1429,P20721,4,2024-05-06T07:04:57Z,0,-36.845165,174.759885,POINT (1756911.566 5920858.442),AF1QipNj6yheGtCvR6Gk2Svq_lG_fuaGPjehPV8kouy8,-36.845177,174.759792,2022-06-24,True,photos:street_view_publish_api,Mint Design,0,low density residential,False,80,0,0,0,2,0,2,1,0


In [3]:
files = glob("nz-*-topo-150k/*.gpkg")
files

['nz-river-polygons-topo-150k/nz-river-polygons-topo-150k.gpkg',
 'nz-lake-polygons-topo-150k/nz-lake-polygons-topo-150k.gpkg',
 'nz-coastlines-topo-150k/nz-coastlines-topo-150k.gpkg',
 'nz-pond-polygons-topo-150k/nz-pond-polygons-topo-150k.gpkg',
 'nz-spring-points-topo-150k/nz-spring-points-topo-150k.gpkg',
 'nz-river-centrelines-topo-150k/nz-river-centrelines-topo-150k.gpkg',
 'nz-swamp-polygons-topo-150k/nz-swamp-polygons-topo-150k.gpkg']

In [4]:
water_dict = {}
for f in tqdm(files):
  name = os.path.dirname(f).replace("nz-", "").replace("-topo-150k", "")
  water_gdf = gpd.read_file(f)
  water_gdf.sindex
  water_dict[name] = water_gdf

  0%|          | 0/7 [00:00<?, ?it/s]

In [5]:
def get_closest_water(point):
  closest_distance = 1e10
  closest_name = "?"
  for name, water_gdf in water_dict.items():
    distance = water_gdf.sindex.nearest(point, return_distance=True)[1][0]
    if distance < closest_distance:
      closest_distance = distance
      closest_name = name
  return closest_name, closest_distance

df["water_type"], df["water_distance"] = zip(*df.geometry.progress_apply(get_closest_water))

  0%|          | 0/1323 [00:00<?, ?it/s]

In [6]:
df.water_type.value_counts()

water_type
river-centrelines    564
lake-polygons        409
coastlines           341
pond-polygons          5
river-polygons         4
Name: count, dtype: int64

In [7]:
df.water_distance.describe().apply("{0:.5f}".format)

count       1323.00000
mean        9402.82600
std       126620.10526
min            4.32646
25%          384.24734
50%          690.59940
75%         1101.74075
max      1885737.33112
Name: water_distance, dtype: object

In [8]:
df.green = df.green.str.extract(r"(\d+)").astype(float)
df.water = df.water.str.extract(r"(\d+)").astype(int)
df.obscured = df.obscured.str.extract(r"(\d+)").astype(int)
df.quality = df.quality.str.extract(r"(\d+)").astype(int)

In [9]:
df.to_csv("merged_results.csv", index=False)

In [10]:
df.environment.value_counts()

environment
low density residential       908
medium density residential    249
commercial                    125
park                           15
shops                          11
indoor                         10
cafes                           4
supermarket                     1
Name: count, dtype: int64

In [11]:
IVs = " + ".join(df.columns[15:])
IVs

'green + environment + active_transport + quality + water + obscured + people + cars + bikes + dwellings + shops + offices + water_type + water_distance'

In [12]:
smf.ols(formula=f"anxiousness ~ {IVs}", data=df).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            anxiousness   R-squared:                       0.024
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     1.374
Date:                Mon, 16 Dec 2024   Prob (F-statistic):              0.111
Time:                        15:57:21   Log-Likelihood:                -3107.8
No. Observations:                1315   AIC:                             6264.
Df Residuals:                    1291   BIC:                             6388.
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
=============================================================================================================
                                                coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------
Intercept                                     0.7753      1.470      0.527      0.598      -2.109       3.660
environment[T.commercial]                     2.0572      1.344      1.531      0.126      -0.579       4.693
environment[T.indoor]                         2.3370      1.558      1.500      0.134      -0.719       5.393
environment[T.low density residential]        2.1825      1.332      1.638      0.102      -0.431       4.796
environment[T.medium density residential]     1.9964      1.340      1.490      0.137      -0.633       4.625
environment[T.park]                           1.7498      1.485      1.178      0.239      -1.164       4.664
environment[T.shops]                          3.7196      1.540      2.416      0.016       0.699       6.740
environment[T.supermarket]                    1.5212      2.917      0.522      0.602      -4.201       7.244
active_transport[T.True]                      0.3774      0.176      2.145      0.032       0.032       0.723
water_type[T.lake-polygons]                  -0.3440      0.196     -1.756      0.079      -0.728       0.040
water_type[T.pond-polygons]                  -0.2802      1.184     -0.237      0.813      -2.604       2.043
water_type[T.river-centrelines]              -0.0674      0.184     -0.366      0.714      -0.429       0.294
water_type[T.river-polygons]                  1.4749      1.313      1.123      0.262      -1.101       4.051
green                                         0.0003      0.004      0.061      0.951      -0.008       0.009
quality                                      -0.0102      0.007     -1.433      0.152      -0.024       0.004
water                                        -0.0018      0.001     -2.032      0.042      -0.004   -6.32e-05
obscured                                      0.0075      0.006      1.278      0.201      -0.004       0.019
people                                        0.0255      0.054      0.477      0.634      -0.079       0.131
cars                                         -0.0161      0.022     -0.739      0.460      -0.059       0.027
bikes                                         0.0220      0.156      0.141      0.888      -0.285       0.329
dwellings                                     0.0166      0.013      1.259      0.208      -0.009       0.042
shops                                         0.0203      0.079      0.255      0.799      -0.136       0.176
offices                                      -0.0993      0.158     -0.628      0.530      -0.409       0.211
water_distance                            -4.851e-07   5.72e-07     -0.847      0.397   -1.61e-06    6.38e-07
==============================================================================
Omnibus:                      130.